In [1]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

# 各パス指定

In [2]:
dataset = 'model/keypoint_classifier/keypoint.csv'
model_save_path = 'model/keypoint_classifier/keypoint_classifier.h5'

# 分類数設定

In [3]:
NUM_CLASSES = 31

# 学習データ読み込み

In [4]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

In [5]:
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

C:\Users\ASUS\AppData\Local\Temp\ipykernel_22140\3345009797.py:1: DeprecationWarning: loadtxt(): Parsing an integer via a float is deprecated.  To avoid this warning, you can:
    * make sure the original data is stored as integers.
    * use the `converters=` keyword argument.  If you only use
      NumPy 1.23 or later, `converters=float` will normally work.
    * Use `np.loadtxt(...).astype(np.int64)` parsing the file as
      floating point and then convert it.  (On all NumPy versions.)
  (Deprecated NumPy 1.23)
  y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

# モデル構築

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [8]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dropout (Dropout)                    │ (None, 42)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 20)                  │             860 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 20)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │             210 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 31)                  │             341 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,411 (5.51 KB)

 Trainable params: 1,411 (5.51 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# モデルチェックポイントのコールバック
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# 早期打ち切り用コールバック
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [10]:
# モデルコンパイル
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# モデル訓練

In [11]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
37/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0264 - loss: 3.4656   
Epoch 1: saving model to model/keypoint_classifier/keypoint_classifier.h5


44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.0278 - loss: 3.4603 - val_accuracy: 0.0866 - val_loss: 3.3482
Epoch 2/1000
34/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0675 - loss: 3.3521 
Epoch 2: saving model to model/keypoint_classifier/keypoint_classifier.h5


44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0691 - loss: 3.3466 - val_accuracy: 0.1219 - val_loss: 3.1894
Epoch 3/1000
35/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0970 - loss: 3.2026 
Epoch 3: saving model to model/keypoint_classifier/keypoint_classifier.h5


44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0969 - loss: 3.1918 - val_accuracy: 0.1267 - val_loss: 2.9528
Epoch 4/1000
34/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1128 - loss: 3.0106 
Epoch 4: saving model to model/keypoint_classifier/keypoint_classifier.h5


44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1139 - loss: 2.9979 - val_accuracy: 0.1662 - val_loss: 2.7206
Epoch 5/1000
36/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1376 - loss: 2.8203 
Epoch 5: saving model to model/keypoint_classifier/keypoint_classifier.h5


...

44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5700 - loss: 1.2256 - val_accuracy: 0.8343 - val_loss: 0.7229
Epoch 291/1000
 1/44 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.6406 - loss: 1.0621
Epoch 291: saving model to model/keypoint_classifier/keypoint_classifier.h5


44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5655 - loss: 1.2427 - val_accuracy: 0.8760 - val_loss: 0.7063
Epoch 292/1000
36/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5482 - loss: 1.2250 
Epoch 292: saving model to model/keypoint_classifier/keypoint_classifier.h5


44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5496 - loss: 1.2277 - val_accuracy: 0.8514 - val_loss: 0.7118
Epoch 293/1000
43/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5458 - loss: 1.2838 
Epoch 293: saving model to model/keypoint_classifier/keypoint_classifier.h5


44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5459 - loss: 1.2837 - val_accuracy: 0.8439 - val_loss: 0.7424
Epoch 294/1000
33/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5676 - loss: 1.2021 
Epoch 294: saving model to model/keypoint_classifier/keypoint_classifier.h5


44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5662 - loss: 1.2104 - val_accuracy: 0.8578 - val_loss: 0.7148
Epoch 294: early stopping


In [12]:
# モデル評価
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8596 - loss: 0.7015 


In [13]:
# 保存したモデルのロード
model = tf.keras.models.load_model(model_save_path)

In [14]:
# 推論テスト
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
[9.09500475e-09 9.76662734e-04 1.35044642e-02 1.60414423e-03
 5.14484286e-01 3.22047770e-02 1.32185742e-02 1.18787475e-02
 1.13115576e-03 2.56683707e-01 2.53823850e-06 3.59958574e-09
 8.71898536e-08 8.36625168e-06 1.25784514e-04 3.02287873e-07
 3.22117433e-02 5.15434798e-03 6.94145216e-04 1.22615776e-03
 1.43148398e-04 1.02300534e-03 1.12512954e-01 1.62184455e-09
 3.46790330e-07 3.99431428e-06 1.03747936e-06 2.00215510e-07
 1.20522873e-03 2.89626936e-08 3.41380471e-08]
4


# 混同行列

In [15]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

def print_confusion_matrix(y_true, y_pred, report=True):
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)
    
    df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)
 
    fig, ax = plt.subplots(figsize=(7, 6))
    sns.heatmap(df_cmx, annot=True, fmt='g' ,square=False)
    ax.set_ylim(len(set(y_true)), 0)
    plt.show()
    
    if report:
        print('Classification Report')
        print(classification_report(y_test, y_pred))

Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)

print_confusion_matrix(y_test, y_pred)

ModuleNotFoundError: No module named 'pandas'

# Tensorflow-Lite用のモデルへ変換

In [ ]:
# 推論専用のモデルとして保存
model.save(model_save_path, include_optimizer=False)

In [ ]:
# モデルを変換(量子化)
tflite_save_path = 'model/keypoint_classifier/keypoint_classifier.tflite'

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

# 推論テスト

In [ ]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

In [ ]:
# 入出力テンソルを取得
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [ ]:
%%time
# 推論実施
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

In [ ]:
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))